# Lab 1: Korean QnA (Question Answering) Training on AWS


## Introduction
---

본 모듈에서는 허깅페이스 트랜스포머(Hugging Face transformers) 라이브러리를 사용하여 한국어 질의응답(Korean QnA; Question Answering) 쌍을 훈련합니다. 질의응답은 본문에서 특정 질의에 대한 답변을 찾는 다운스트림 태스크입니다. 데이터는 본문(context)/질의(question) 쌍의 형태로 제공되며, 정답은 본문 내에 포함된 정답(answer) 문구의 시작과 끝을 색인화합니다. 

***[Note] SageMaker Studio Lab, SageMaker Studio, SageMaker 노트북 인스턴스, 또는 여러분의 로컬 머신에서 이 데모를 실행할 수 있습니다. SageMaker Studio Lab을 사용하는 경우 GPU를 활성화하세요.***

***[주의] 본 데이터셋은 상업적인 목적으로 사용할 수 없습니다. 본 핸즈온은 연구/참고용으로만 활용하시고, 모델 훈련은 여러분만의 데이터셋을 직접 생성하셔야 합니다.***

### References
- Hugging Face Tutorial: https://huggingface.co/docs/transformers/tasks/question_answering
- Fine-tuning with custom datasets: https://huggingface.co/transformers/v4.11.3/custom_datasets.html#question-answering-with-squad-2-0
- KorQuAD 1.0: https://korquad.github.io/KorQuad%201.0/


## 1. Setup Environments
---

### Import modules

In [1]:
import os
import sys
import json
import logging
import argparse
import torch
from torch import nn
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import (
    BertTokenizer, BertTokenizerFast, BertConfig, BertForTokenClassification, 
    Trainer, TrainingArguments, set_seed
)
from transformers.trainer_utils import get_last_checkpoint

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

### Argument parser

In [2]:
def parser_args(train_notebook=False):
    parser = argparse.ArgumentParser()

    # Default Setting
    parser.add_argument("--epochs", type=int, default=3)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--train_batch_size", type=int, default=32)
    parser.add_argument("--eval_batch_size", type=int, default=64)
    parser.add_argument("--max_length", type=int, default=384)
    parser.add_argument("--stride", type=int, default=64)
    parser.add_argument("--warmup_steps", type=int, default=100)
    parser.add_argument("--logging_steps", type=int, default=100)
    parser.add_argument("--learning_rate", type=str, default=5e-5)
    parser.add_argument("--disable_tqdm", type=bool, default=False)
    parser.add_argument("--fp16", type=bool, default=True)
    parser.add_argument("--tokenizer_id", type=str, default='salti/bert-base-multilingual-cased-finetuned-squad')
    parser.add_argument("--model_id", type=str, default='salti/bert-base-multilingual-cased-finetuned-squad')
    
    # SageMaker Container environment
    parser.add_argument("--output_data_dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--model_dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--n_gpus", type=str, default=os.environ["SM_NUM_GPUS"])
    parser.add_argument("--train_dir", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--valid_dir", type=str, default=os.environ["SM_CHANNEL_VALID"])
    parser.add_argument('--chkpt_dir', type=str, default='/opt/ml/checkpoints')     

    if train_notebook:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()
    return args

In [3]:
train_dir = 'qna_train'
valid_dir = 'qna_valid'
!rm -rf {train_dir} {valid_dir}
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True) 

### Load Arguments

주피터 노트북에서 곧바로 실행할 수 있도록 설정값들을 로드합니다. 물론 노트북 환경이 아닌 커맨드라인에서도 `cd scripts & python3 train.py` 커맨드로 훈련 스크립트를 실행할 수 있습니다.

In [4]:
chkpt_dir = 'chkpt'
model_dir = 'model'
output_data_dir = 'data'
num_gpus = torch.cuda.device_count()

!rm -rf {chkpt_dir} {model_dir} {output_data_dir} 

if os.environ.get('SM_CURRENT_HOST') is None:
    is_sm_container = False

    #src_dir = '/'.join(os.getcwd().split('/')[:-1])
    src_dir = os.getcwd()
    os.environ['SM_MODEL_DIR'] = f'{src_dir}/{model_dir}'
    os.environ['SM_OUTPUT_DATA_DIR'] = f'{src_dir}/{output_data_dir}'
    os.environ['SM_NUM_GPUS'] = str(num_gpus)
    os.environ['SM_CHANNEL_TRAIN'] = f'{src_dir}/{train_dir}'
    os.environ['SM_CHANNEL_VALID'] = f'{src_dir}/{valid_dir}'

args = parser_args(train_notebook=True) 
args.chkpt_dir = chkpt_dir
logger.info("***** Arguments *****")
logger.info(''.join(f'{k}={v}\n' for k, v in vars(args).items()))

os.makedirs(args.chkpt_dir, exist_ok=True) 
os.makedirs(args.model_dir, exist_ok=True)
os.makedirs(args.output_data_dir, exist_ok=True) 

[{204499775.py:21} INFO - ***** Arguments *****
[{204499775.py:22} INFO - epochs=3
seed=42
train_batch_size=32
eval_batch_size=64
max_length=384
stride=64
warmup_steps=100
logging_steps=100
learning_rate=5e-05
disable_tqdm=False
fp16=True
tokenizer_id=salti/bert-base-multilingual-cased-finetuned-squad
model_id=salti/bert-base-multilingual-cased-finetuned-squad
output_data_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/question-answering/data
model_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/question-answering/model
n_gpus=4
train_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/question-answering/qna_train
valid_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/question-answering/qna_valid
chkpt_dir=chkpt



<br>

## 2. Preparation
---

### Dataset

본 핸즈온에서 사용할 데이터셋은 KorQuAD (The Korean Question Answering Dataset) 1.0 입니다. 1,560개의 한국어 위키피디아 글들에 대해 66,181개의 질의응답 쌍(60,407개 훈련 데이터, 5,774개 검증 데이터)으로 구성된 데이터셋으로 SQuAD (Stanford Question Answering Dataset) 1.0과 동일한 포맷입니다.
- KorQuad 1.0: https://korquad.github.io/KorQuad%201.0/

In [5]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O {train_dir}/KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O {valid_dir}/KorQuAD_v1.0_dev.json

--2022-07-06 04:57:46--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘qna_train/KorQuAD_v1.0_train.json’

100%[======================================>] 38,527,475  --.-K/s   in 0.1s    

2022-07-06 04:57:46 (269 MB/s) - ‘qna_train/KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2022-07-06 04:57:46--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.111.153, 185.199.108.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘qna_valid/KorQuAD_v1.0_dev.json’

1

<br>

## 3. Construct Feature set
---

질의응답쌍에 대한 모델을 훈련하려면 토큰화된 본문/질문 쌍과 답변 범위를 알 수 있는 토큰 인덱스가 필요합니다. 


### Load raw data

원본 데이터로부터 본문 및 질의응답 정보를 로드합니다.

In [6]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad(f'{train_dir}/KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad(f'{valid_dir}/KorQuAD_v1.0_dev.json')

### Add End index

문장에서 답변이 끝나는 문자 위치를 계산합니다. 때로, 답변이 한두 글짜씩 차이가 나는 경우가 있으므로 이에 대한 예외처리를 추가합니다.

In [7]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters
    return answers

train_answers = add_end_idx(train_answers, train_contexts)
val_answers = add_end_idx(val_answers, val_contexts)

### Tokenization 

연어 처리 모델을 훈련하려면, 토큰화(Tokenization)를 통해 말뭉치(corpus; 자연어 처리를 위한 대량의 텍스트 데이터)를 토큰 시퀀스로 나누는 과정이 필요합니다. BERT 이전의 자연어 처리 모델은 주로 도메인 전문가들이 직접 토큰화해놓은 토크아니저(Mecab, Kkma 등)들을 사용했지만, BERT를 훈련하기 위한 토크나이저는 도메인 지식 필요 없이 말뭉치에서 자주 등장하는 서브워드(subword)를 토큰화합니다. GPT 기반 모델은 BPE(Byte-pair Encoding)라는 통계적 기법을 사용하며, BERT 및 ELECTRA 기반 모델은 BPE와 유사한 Wordpiece를 토크나이저로 사용합니다.

In [8]:
from transformers import AutoTokenizer, BertTokenizerFast
#tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_id)
tokenizer = BertTokenizerFast.from_pretrained(args.tokenizer_id)

train_encodings = tokenizer(
    train_contexts, 
    train_questions, 
    truncation=True, 
    max_length=args.max_length,
    stride=args.stride, 
    padding="max_length"
)
val_encodings = tokenizer(
    val_contexts, 
    val_questions, 
    truncation=True, 
    max_length=args.max_length,
    stride=args.stride, 
    padding="max_length"
)

### Covert to token positions for answers

답변의 시작/끝 인덱스를 토큰 시작/끝 인덱스로 변환합니다. 원래의 문자열에서 토큰 인덱스를 반환하는 `char_to_token()` 메소드로 편하게 수행할 수 있습니다.

In [9]:
def add_token_positions(encodings, answers, tokenizer):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            
        # If the start and end positions are greater than max_length, both must be changed to max_length.
        if start_positions[-1] is None or start_positions[-1] > tokenizer.model_max_length:
            start_positions[-1] = tokenizer.model_max_length
        
        if end_positions[-1] is None or end_positions[-1] > tokenizer.model_max_length:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})   
    return encodings

train_encodings = add_token_positions(train_encodings, train_answers, tokenizer)
val_encodings = add_token_positions(val_encodings, val_answers, tokenizer)

### Custom Dataset

훈련/검증 시에 사용할 커스텀 데이터셋을 생성하기 위한 클래스를 생성합니다. BERT 질의응답쌍 모델은 보통 아래의 입력값을 사용합니다.
- `input_ids`: 문장이 인덱스(특정 vocab에 매핑하는 숫자값)로 구성된 토큰 시퀀스로 변환된 결괏값
- `attention_mask` : 해당 토큰이 패딩 토큰인지, 아닌지를 마스킹
- `token_type_ids`: 세그먼트 (두 문장 입력 시, 첫번째 문장인지 아닌지를 마스킹)
- `start_positions`: 답변이 시작하는 토큰 위치
- `end_positions`: 답변이 끝나는 토큰 위치

In [10]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

<br>

## 4. Training
---

### Training Preparation

본 핸즈온은 허깅페이스의 트랜스포머 라이브러리에 포함된 BertForQuestionAnswering 모델을 사용합니다.

In [11]:
from transformers import (
    BertForQuestionAnswering,
    Trainer, TrainingArguments, set_seed
)
from transformers.trainer_utils import get_last_checkpoint

model = BertForQuestionAnswering.from_pretrained(args.model_id)
training_args = TrainingArguments(
    output_dir='chkpt',
    overwrite_output_dir=True if get_last_checkpoint('chkpt') is not None else False,
    num_train_epochs=args.epochs,
    per_device_train_batch_size=args.train_batch_size, 
    per_device_eval_batch_size=args.eval_batch_size, 
    warmup_steps=args.warmup_steps, 
    weight_decay=0.01,    
    logging_dir="logs", 
    logging_steps=args.logging_steps,
    learning_rate=float(args.learning_rate),
    save_total_limit=5,
    save_strategy="epoch",
    fp16=args.fp16,
    gradient_accumulation_steps=4,
    #evaluation_strategy="steps",
)

훈련을 수행하기 위한 `Trainer` 클래스를 인스턴스화합니다.

In [12]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

Using cuda_amp half precision backend


### Training
훈련을 수행합니다. 딥러닝 기반 자연어 처리 모델 훈련에는 GPU가 필수이며, 본격적인 훈련을 위해서는 멀티 GPU 및 분산 훈련을 권장합니다. 만약 멀티 GPU가 장착되어 있다면 Trainer에서 총 배치 크기 = 배치 크기 x GPU 개수로 지정한 다음 데이터 병렬화를 자동으로 수행합니다.

In [13]:
%%time
# train model
if get_last_checkpoint(args.chkpt_dir) is not None:
    logger.info("***** Continue Training *****")
    last_checkpoint = get_last_checkpoint(args.chkpt_dir)
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 60407
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 4
  Total optimization steps = 354
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
100,1.463100
200,0.481400
300,0.364300


Saving model checkpoint to chkpt/checkpoint-118
Configuration saved in chkpt/checkpoint-118/config.json
Model weights saved in chkpt/checkpoint-118/pytorch_model.bin
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to chkpt/checkpoint-236
Configuration saved in chkpt/checkpoint-236/config.json
Model weights saved in chkpt/checkpoint-236/pytorch_model.bin
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to chkpt/checkpoint-354
Configuratio

CPU times: user 41min 50s, sys: 5min 59s, total: 47min 50s
Wall time: 17min 54s


In [14]:
tokenizer.save_pretrained(args.model_dir)                
trainer.save_model(args.model_dir)

tokenizer config file saved in /home/ec2-user/SageMaker/sm-kornlp-usecases/question-answering/model/tokenizer_config.json
Special tokens file saved in /home/ec2-user/SageMaker/sm-kornlp-usecases/question-answering/model/special_tokens_map.json
Saving model checkpoint to /home/ec2-user/SageMaker/sm-kornlp-usecases/question-answering/model
Configuration saved in /home/ec2-user/SageMaker/sm-kornlp-usecases/question-answering/model/config.json
Model weights saved in /home/ec2-user/SageMaker/sm-kornlp-usecases/question-answering/model/pytorch_model.bin


<br>

## 5. Evaluation
---

평가를 수행합니다. 질의응답 태스크의 평가는 EM(Exact Match)와 F1 score를 사용합니다.
- EM: 예측한 정답의 토큰이 정확히 일치하면 1, 그렇지 않으면 0
- F1: 예측한 정답과 실제 정답의 중첩 토큰으로 스코어 산출

Reference: https://qa.fastforwardlabs.com/no%20answer/null%20threshold/bert/distilbert/exact%20match/f1/robust%20predictions/2020/06/09/Evaluating_BERT_on_SQuAD.html

In [15]:
# compute metrics (EM and F1)
from scripts.evaluate import get_metrics_korquadv1, get_prediction
em, f1 = get_metrics_korquadv1(args.valid_dir, tokenizer, model)
logger.info(f"EM = {em}")
logger.info(f"F1 = {f1}")

100%|██████████| 5774/5774 [01:00<00:00, 96.18it/s]

[{3242478017.py:4} INFO - EM = 0.627641149982681
[{3242478017.py:5} INFO - F1 = 0.6994331158010804


### Example
여러분만의 샘플 문장을 만들어서 자유롭게 추론을 수행해 보세요.

In [16]:
from transformers import pipeline
nlp = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

context = r"""
아마존웹서비스(AWS)는 카카오 게임 전문 계열사 카카오게임즈가 자사 머신러닝(ML), 데이터베이스(DB) 및 데이터 분석 등 서비스를 통해 사용자 경험을 제고했다고 7일 밝혔다.
AWS는 카카오게임즈가 AWS클라우드 역량을 활용해 게임 데이터 분석 솔루션을 실행하고, 대량의 게임 데이터와 설치 건수, 사용자 유지율과 같은 성과 지표를 분석하고 있다고 설명했다. 
현재 카카오게임즈는 폭증하는 데이터를 저장·분석하기 위한 방법으로 클라우드 오브젝트 스토리지 서비스 '아마존 S3(Amazon Simple Storage Service)' 기반 데이터 레이크(Data Lake)를 구축했다. 또 데이터 분석을 용이하게 해주는 대화형 쿼리 서비스 '아마존 아테나(Amazon Athena)'를 도입해 데이터 레이크로부터 게임 데이터를 통합하고, 게임 사용자 행동과 관련된 인사이트를 확보 중이다. 
이를 통해 카카오게임즈는 게임 봇을 탐지하고 제거하는 방식으로 사용자 경험을 제고했다. 또한 관계형 데이터베이스 서비스 '아마존 오로라(Amazon Aurora)'를 활용해 게임 내 구매와 같은 대규모 데이터베이스 거래를 처리하고 있다. 이밖에도 카카오게임즈는 ML 모델 구축, 교육 및 배포를 위한 완전 관리형 서비스 '아마존 세이지메이커(Amazon SageMaker)'를 활용할 예정이다.
"""

print(nlp(question="카카오 게임 전문 계열사는?", context=context))
print(nlp(question="AWS의 클라우드 오브젝트 스토리지 서비스는?", context=context))
print(nlp(question="AWS의 관계형 데이터베이스 서비스는?", context=context))
print(nlp(question="AWS의 ML 모델 완전 관리형 서비스는?", context=context))

{'score': 0.9967060089111328, 'start': 29, 'end': 36, 'answer': '카카오게임즈가'}
{'score': 0.8259254097938538, 'start': 263, 'end': 269, 'answer': '아마존 S3'}
{'score': 0.879924476146698, 'start': 514, 'end': 521, 'answer': '아마존 오로라'}
{'score': 0.9202089905738831, 'start': 626, 'end': 636, 'answer': '아마존 세이지메이커'}
